## Least Cost Pipeline Construction - sorted on potential
This notebook constructs a pipeline network connecting all biogas sources using the following workflow: 
1.	Start with the biogas source with the largest potential (f^3/yr) and find its least cost path to the existing pipeline network.
2.	Move to the site with the next highest potential and find its least cost path to the updated pipeline (original + the pipeline linking the 1st site to the original pipe network). 
3.	Proceed with remaining sites, in order of declining potential, until all are connected.
4.	Then, when all sites have been connected, compute the accumulated volume of biogas passing through each segment from all “upstream sites”. 

#### The datasets required for this analysis include:
* The spreadsheet of biogas source, incliding coordinates and biogas potential 
* The MIT pipeline cost surface (all US)
* The REXTAG natural gas pipeline raster (all US)

#### Analysis consists of the following functions:
* **Data Prep**:
 * Import biogas locations as Pandas dataframe: `df_biogas`
 * Convert biogas locations to GeoPandas geodataframe, using WGS 84 coordinate ref. system (CRS): `gdf_biogas`
 * Import MIT pipeline cost surface as rasterio raster: `ds_pipecost_full`
 * Transform biogas geodataframe to same CRS as pipeline cost raster: `ds_pipelines_full`
 * Subset pipeline cost raster to extent of biogas locations, buffered by 5km: `ds_pipecost`
 * Subset pipeline raster to same extent as above: `ds_pipelines`

* **Functions**:
 * `cost_distance(source_pt, cost_array)`: Compute cost distance away from site. Outputs are a cost distance and traceback array. 
 * `compute_lcp(source_pt, cost_array, pipes_array)`: 

---
## Data Prep

In [174]:
#Imports
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point,LineString
import rasterio
import rasterio.mask

from skimage import graph

import matplotlib.pyplot as plt
%matplotlib inline

* Filenames

In [64]:
#Inputs
fn_biogas_excel = '..\\data\\NC_Biogas_Potential.xlsx'
fn_MIT_cost_surface = '..\\data\\MIT_Surface_Full\\costsurface500m.img'
fn_REXTAG_pipe_raster = '..\\data\\processed\\USA_pipes.tif'

#Outputs
fn_biogas_points = '..\\data\\processed\\biogas_sources.shp'
fn_subset_rasters =  '..\\data\\processed\\subset_data.tif'

* Import biogas data and convert to geodataframe

In [10]:
#Read the excel sheet into a dataframe
df_biogas = pd.read_excel(fn_biogas_excel,sheet_name='Swine')
#Create a series of Point geometries for each record
geom = [Point(xy) for xy in zip(df_biogas.Longitude, df_biogas.Latitude)]
#Construct the geodataframe from the data and geometry, setting crs to WGS84
gdf_biogas = gpd.GeoDataFrame(df_biogas, geometry = geom, crs = 4326)
#Delete the original dataframe (to conserve memory)
del(df_biogas)

* Import the MIT cost and REXTAG pipelines rasters

In [19]:
#Read the MIT img file into a rasterio raster
ds_pipecost_full = rasterio.open(fn_MIT_cost_surface)
#Read the REXTAG tif file into a rasterio raster
ds_pipelines_full = rasterio.open(fn_REXTAG_pipe_raster)

* Transform the biogas points to the same crs as the MIT raster

In [66]:
#Get the MIT coordinate reference system
crs_pipecost = ds_pipecost_full.crs
#Apply the transformation
gdf_biogas = gdf_biogas.to_crs(crs_pipecost)
#Save to file
gdf_biogas.to_file(fn_biogas_points)

* Subset the cost and pipe rasters to the extent of the points + 5km

In [34]:
#Collapse the biogas points to a multipoint object, buffer 5000m, and pull its exent
bg_extent = gdf_biogas.geometry.unary_union.buffer(5000).envelope

#Mask the cost and pipe rasters
cost_image, cost_transform = rasterio.mask.mask(ds_pipecost_full,[bg_extent],crop=True)
pipe_image, pipe_transform = rasterio.mask.mask(ds_pipelines_full,[bg_extent],crop=True)

#Update the pipe raster dtype (uint8) to match the cost raster's (uint16)
pipe_image = pipe_image.astype(cost_image.dtype)

* Save cost and pipeline subset images to two band raster dataset

In [62]:
#Extract the dimensions of the output rasters
out_height = cost_image.shape[1]
out_width = cost_image.shape[2]
#Create the subset cost surface metadata (altered from original)
out_meta = ds_pipecost_full.meta
out_meta.update({"driver":"GTiff","height":out_height,"width":out_width,"count":2,"transform":cost_transform})
#Stack the two images and write to output file
out_stacked = np.concatenate((cost_image,pipe_image),axis=0)
with rasterio.open(fn_subset_rasters,'w',**out_meta) as dst:
    dst.write(out_stacked)

In [88]:
#Read the raster back in 
ds_subset = rasterio.open(fn_subset_rasters)

---
## Analysis: Functions
1. Compute least cost path for a biogas source to nearest pipe

In [122]:
#Convert pipes array to a masked array
arr_pipes = ds_subset.read(2)
pipe_mask = np.ma.masked_array(arr_pipes,mask=arr_pipes==0)

In [93]:
#Create a graph from the cost raster
x_size,y_size = ds_subset.res
arr_cost = ds_subset.read(1)
lc_graph = graph.MCP_Geometric(arr_cost,sampling=(x_size,y_size))

In [189]:
#def getLCP(layer_number)
layer_number = 1

#Get the biogas feature for the specified layer number
bg_point = gdf_biogas.loc[layer_number,'geometry']
#Get the index location
idx = ds_subset.index(bg_point.x,bg_point.y)
#Compute the cost-distance and traceback arrays
cd_array, tb_array = lc_graph.find_costs(starts=([idx]))

In [157]:
##Find the pipe coord with lowest cost in cd_array

#Mask just pipeline pixels from cost distance output
arr_cd_pipes = np.ma.masked_array(cd_array,mask=arr_pipes==0)
#Locate the min value
minPipeCost = arr_cd_pipes.min()
#Determine the row and column where the min occurs
rMin, cMin = np.where(arr_cd_pipes == minPipeCost)
#Extract values from value arrays
pipe_coords = (rMin[0],cMin[0])

In [158]:
#Get the row/col indices of pixels in the LCP 
lcp_indices = lc_graph.traceback(pipe_coords)

In [195]:
#Create linestring from lcp_indices
pipe_line = LineString([ds_subset.xy(r,c) for r,c in lcp_indices])
#Append pipe_line to gdf
gdf_biogas.loc[layer_number,'route'] = pipe_line

In [182]:
#Convert lcp_indices to array of ones (pipes) and zeros
arr_LCP = np.zeros(arr_cost.shape)
for r,c in lcp_indices:
    arr_LCP[r,c] = 1